### Prep D - Generate Origins for 2016

Part of the complexity of the Yemen anaylsis is that we want to compare the access to helath facilities across two different time periods - 2016 and 2018. 

Unfortunately, WorldPop supplied us with grids which were slightly offset for the two years. This is not good for the purposes of network analysis - as we want to the origin points to remain constant (geographically) in order to do change detection between the resultant travel time rasters. 

As such, it was necessary to devise a means of translating the values on the 2016 file to the geographical centroids of the 2018 file to carry out the analysis. This is what this script accomplishes.

We commence per normal by importing the usual suspects. 

In [ ]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST\GOSTRocks')
import rasterMisc as rm
import rasterio as rt
import geopandas as gpd
from shapely.geometry import box, Point

Firstly, we import the 2018 points, and build a list of nodes composed of x,y coordinate pairs

In [98]:
origin_pth = r'C:\Users\charl\Documents\GOST\Yemen\origins'
points_2018 = gpd.read_file(os.path.join(origin_pth, 'origins_1km_2018.shp'))
points_2018['x'] = points_2018.geometry.x
points_2018['y'] = points_2018.geometry.y

list_of_nodes = {}
for index, row in points_2018.iterrows():
    list_of_nodes.update({index:(row['x'], row['y'])})

We only have population for 2015, and facility availability for 2016. Rather than moan about it, we get on and use the 2015 population layer for 2016. As usual, we start by opening the file

In [ ]:
pop_2015 = rt.open(os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\worldpop',r'pop15_resampled.tif'))

We want to sample the raster for only the points that fall inside raster. So we do a quick intersection check to build a list of the nodes that definitely intersect the bounds of the raster. 

In [ ]:
# take bounds
b = pop_2015.bounds

# build bounding box of the raster
datasetBoundary = box(b[0], b[1], b[2], b[3])

# iterate through the nodes list
selKeys = []
selPts = []
for key, pt in list_of_nodes.items():
    
    # Make a point, check if it lies inside the bounding box just defined
    if Point(pt[0], pt[1]).intersects(datasetBoundary):
        
        # if so, add the Point and keys to the respective lists
        selPts.append(pt)
        selKeys.append(key)

Now, we go ahead and sample the 2015 raster with these points. 

In [99]:
raster_values = list(pop_2015.sample(selPts))
raster_values = [x[0] for x in raster_values]

# generate new dictionary of {node ID: raster values}
ref = dict(zip(selKeys, raster_values))

Before mapping the values back on to original 2018 points using the pandas map function on the ref dictionary of ID:value pairs

In [106]:
# this is the key line that translates the index field into the 2015 population value
points_2018['2015_POP'] = points_2018.index.map(ref)

# we reset this field as 'VALUE' and drop annoying fields
points_2018['VALUE'] = points_2018['2015_POP']
points_2018 = points_2018.drop(['x','y','2015_POP'], axis = 1)

At this point we are ready to write out the 2015 layer as both shapefile and .csv, which we do below.

In [110]:
points_2018.to_file(os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\origins','origins_1km_2015.shp'), driver = 'ESRI Shapefile')
points_2018.to_csv(os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\origins','origins_1km_2015.csv'))